# Ćwiczenie 3

Celem ćwiczenia jest imlementacja metody [Minimax z obcinaniem alpha-beta](https://en.wikipedia.org/wiki/Alpha%E2%80%93beta_pruning) do gry Connect Four (czwórki).

W trakcie ćwiczenia można skorzystać z reposytorium z implementacją gry [Connect Four udostępnionym przez Jakuba Łyskawę](https://github.com/lychanl/two-player-games). Ewentualnie, można zaimplementować samemu grę Connect Four (ale, tak aby rozwiązanie miało ten sam interfejs co podany poniżej).

Implementację Minimax należy przetestować używając różną głębokość przeszukiwania. Implementacja Solvera musi zapewniać interfejs jak poniżej, ale można dodać dowolne metody prywatne oraz klasy wspomagające (jeżeli będą potrzebne).

Punktacja:
- Działająca metoda Minimax - **2 pkt**
- Działająca metoda Minimax z obcinaniem alpha-beta - **1.5 pkt**
- Analiza jakości solvera w zależności od głębokości przeszukiwania **1.5pkt**
    - należy zaimplementować w tym celu prostą wizualizację rozgrywki dwóch agentów, bądź kilka przykładów 'z ręki'
- Jakość kodu **2pkt**

Aby importowanie elementów z poniższej komórki działało należy umieścić tego notebooka w tym samym folderze co paczkę `two_player_games`:
```
├── LICENSE
├── README.md
├── minimax.ipynb # <<< HERE
├── test
│   ├── __init__.py
│   ├── test_connect_four.py
│   ├── test_dots_and_boxes.py
│   └── test_pick.py
└── two_player_games
    ├── __init__.py
    ├── games
    │   ├── connect_four.py
    │   └── dots_and_boxes.py
    ├── move.py
    ├── player.py
    └── state.py
```

In [14]:
from typing import Tuple, List

from two_player_games.player import Player
from two_player_games.games.connect_four import ConnectFour, ConnectFourMove, ConnectFourState
from copy import copy
from random import choice
import numpy as np
import time
import ipywidgets as widgets

from connect4_solvers import Solver, RandomSolver, HeuristicSolver, MinMaxSolver

Wielkość planszy

In [15]:
ROW_COUNT = 6
COLUMN_COUNT = 7

In [35]:
def simulate_move(board, column):
    board_copy = copy(board)
    board_copy = board_copy.make_move(ConnectFourMove(column))
    return board_copy

"""class MinMaxSolver:
    def __init__(self, game: ConnectFour, row_count: int, column_count: int, max_player: Player, min_player: Player):
        self.game = game
        self._row_count = row_count
        self._column_count = column_count

        self._max_player = max_player
        self._min_player = min_player
        
        self._prizes = {
            "plr_two_in_seg": 5,
            "plr_three_in_seg": 10,
            "plr_winning_cond": 20,
            "opp_winning_cond": 80
        }

        total_weight = sum(self._prizes.values())
        self._prizes = {key: value / total_weight for key, value in
                        self._prizes.items()}

    def evaluate_position(self, board: ConnectFourState,
                          player: Player) -> float:
        prize = 0

        # Check verticals
        for i in range(self._column_count):
            for j in range(self._row_count - 3):
                segment = [board.fields[i][j + z] for z in range(4)]
                prize += self._evaluate_segment(segment, player)

        # Check horizontals
        for i in range(self._column_count - 3):
            for j in range(self._row_count):
                segment = [board.fields[i + z][j] for z in range(4)]
                prize += self._evaluate_segment(segment, player)

        # Check rising edge diagonals
        for i in range(self._column_count - 3):
            for j in range(self._row_count - 3):
                segment = [board.fields[i + z][j + z] for z in range(4)]
                prize += self._evaluate_segment(segment, player)

        # Check falling edge diagonals
        for i in range(self._column_count - 3):
            for j in range(3, self._row_count):
                segment = [board.fields[i + z][j - z] for z in range(4)]
                prize += self._evaluate_segment(segment, player)

        return prize

    def _evaluate_segment(self, segment: List, player: Player):
        prize = 0

        # Evaluate max player and min player possibility
        # for winning combination in the future
        if segment.count(self._max_player) == 1 and segment.count(None) == 3:
            prize += 1
        elif segment.count(self._min_player) == 1 and segment.count(None) == 3:
            prize -= 1

        # Evaluate already existing combinations
        if segment.count(self._max_player) == 4:
            prize += self._prizes["plr_winning_cond"]
        elif segment.count(self._max_player) == 3 and segment.count(None) == 1:
            prize += self._prizes["plr_three_in_seg"]
        elif segment.count(self._max_player) == 2 and segment.count(None) == 2:
            prize += self._prizes["plr_two_in_seg"]
        elif segment.count(self._min_player) == 3 and segment.count(None) == 1:
            prize -= self._prizes["opp_winning_cond"]

        return prize
    
    def is_valid_move(self, col_index: int) -> bool:
        # Check if move to column is available
        if self.game.state.fields[col_index][-1] is None:
            return True
        else:
            return False

    def get_valid_moves(self):
        # Return list of all currently available moves
        return [valid_column for valid_column in range(self._column_count) if
                self.is_valid_move(valid_column)]


    def get_best_move(self, depth, alpha_beta=True, *args):
        if alpha_beta:
            col, score = self.minimax_alpha_beta(self.game.state, depth,
                                                 -np.inf, np.inf,
                                                 True)
            return col
        else:
            col, score = self.minimax(self.game.state, depth, True)
            return col

    def minimax_alpha_beta(self, board, depth, alpha: float, beta: float,
                is_maximizing_player: bool) -> Tuple[int, float]:
        # Returns column index and score
        valid_moves = self.get_valid_moves()
        is_terminal = board.is_finished()

        if is_terminal or depth == 0:
            if is_terminal:
                if board.get_winner() == self._max_player:
                    return (None, 1e10)
                elif board.get_winner() == self._min_player:
                    return (None, -1e10)
                else:
                    return (None, 0)
            else:
                return (
                    None, self.evaluate_position(board, self._max_player))

        if is_maximizing_player:
            value = -np.inf
            chosen_column = choice(valid_moves)
            for valid_move in valid_moves:
                evaluation = self.minimax_alpha_beta(simulate_move(board, valid_move), depth - 1, alpha, beta, False)[1]

                if evaluation > value:
                    value = evaluation
                    chosen_column = valid_move

                alpha = max(alpha, value)

                if alpha >= beta:
                    break

            return chosen_column, value

        else:
            value = np.inf
            chosen_column = choice(valid_moves)
            for valid_move in valid_moves:
                evaluation = self.minimax_alpha_beta(simulate_move(board, valid_move), depth - 1, alpha, beta, True)[1]

                if evaluation < value:
                    value = evaluation
                    chosen_column = valid_move

                beta = min(beta, value)

                if alpha >= beta:
                    break

            return chosen_column, value

    def minimax(self, board, depth, is_maximizing_player: bool) -> Tuple[int, float]:
        # Returns column index and score
        valid_moves = self.get_valid_moves()
        is_terminal = board.is_finished()

        if is_terminal or depth == 0:
            if is_terminal:
                if board.get_winner() == self._max_player:
                    return (None, 1e10)
                elif board.get_winner() == self._min_player:
                    return (None, -1e10)
                else:
                    return (None, 0)
            else:
                return (
                    None, self.evaluate_position(board, self._max_player))

        evaluations = []
        for valid_move in valid_moves:
            evaluation = self.minimax(simulate_move(board, valid_move), depth - 1, not is_maximizing_player)[1]
            evaluations.append((valid_move, evaluation))

        if is_maximizing_player:
            moves = [score for score in evaluations if
                     score[1] == max([score[1] for score in evaluations])]

            return choice(moves)

        else:
            moves = [score for score in evaluations if
                     score[1] == min([score[1] for score in evaluations])]

            return choice(moves)
"""
pass
        

# Symulacje Rozgrywki

In [17]:
def make_stats(p1: Player, p2: Player, row_count: int, column_count: int, solver1: Solver, solver2: Solver,
               solver1_args: List, solver2_args: List, games: int):
    # Dobre głębokości przeszukiwań
    # 2, 4, 6, 8, 10

    p1_won = 0
    p2_won = 0
    for game_nb in range(1, games+1):
        game = ConnectFour(size=(column_count, row_count), first_player=p1,
                           second_player=p2)
        solver1.game = game
        solver2.game = game
        i = 0

        while not game.state.is_finished():
            time.sleep(0.05)

            p1_best_move = solver1.get_best_move(*solver1_args)
            game.make_move(ConnectFourMove(p1_best_move))

            if game.state.is_finished():
                break

            i += 1
            p2_best_move = solver2.get_best_move(*solver2_args)
            game.make_move(ConnectFourMove(p2_best_move))

        print(game)
        print(f"Game: {game_nb}/{games}")
        print(f"Moves: {i}")

        if game.get_winner() is not None:
            if game.get_winner().char == p1.char:
                print(f"Won: {game.get_winner().char}")
                p1_won += 1
            elif game.get_winner().char == p2.char:
                print(f"Won: {game.get_winner().char}")
                p2_won += 1

    draws = games - p1_won - p2_won
    print("Stats")
    print(
        f"Games: {games}\t Draws: {draws}\tDraws percent: {round((draws / games) * 100, 2)}")
    print(
        f"P1 winnings stats\t Won: {p1_won}\tLoosed: {p2_won} \tWin percent: {round((p1_won / games) * 100, 2)}")
    print(
        f"P1 winnings stats\t Won: {p2_won}\tLoosed: {p1_won} \tWin percent: {round((p2_won / games) * 100, 2)}")

In [18]:
p1 = Player("x")
p2 = Player("o")

# Rozgrywka agenta losowego z agentem losowym

In [19]:
algorithm = RandomSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p1, min_player=p2)
algorithm2 = RandomSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p1, min_player=p2)
algorithm_args = []
algorithm2_args = []

make_stats(p1, p2, ROW_COUNT, COLUMN_COUNT, algorithm, algorithm2, algorithm_args, algorithm2_args, games=1000)

Current player: o
[o][ ][ ][ ][ ][ ][ ]
[x][ ][ ][ ][ ][ ][ ]
[x][ ][ ][ ][ ][x][o]
[o][ ][o][ ][ ][x][x]
[x][x][o][x][x][x][o]
[o][o][o][x][o][x][o]
Game: 1/1000
Moves: 11
Won: x
Current player: o
[ ][ ][ ][o][ ][ ][o]
[ ][ ][ ][x][ ][x][x]
[ ][x][ ][x][x][o][o]
[ ][o][ ][o][o][x][x]
[x][o][x][o][x][o][x]
[o][o][x][x][o][x][o]
Game: 2/1000
Moves: 14
Won: x
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[x][ ][ ][ ][ ][o][o]
[x][x][ ][x][x][o][o]
[o][o][o][o][x][x][x]
[x][x][o][x][o][o][o]
[x][o][x][o][x][o][x]
Game: 3/1000
Moves: 15
Won: o
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][o][ ][ ][x]
[o][ ][ ][o][ ][ ][x]
[o][x][ ][o][x][x][x]
Game: 4/1000
Moves: 6
Won: o
Current player: o
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[x][ ][ ][ ][ ][x][ ]
[o][ ][ ][ ][o][x][o]
[x][ ][o][o][o][x][o]
[x][x][x][x][o][o][x]
Game: 5/1000
Moves: 9
Won: x
Current player: o
[ ][ ][ ][o][x][ ][ ]
[ ][ ][ ][o][x][ ][ ]
[x][ ][o][x][x][x][ ]
[o][ ][o][o][o][x]

# Rozgrywka agenta losowego z agentem wykorzystującym czystą funkcję heurystyczną

In [30]:
algorithm = RandomSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p1, min_player=p2)
algorithm2 = HeuristicSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p2, min_player=p1)
algorithm_args = []
algorithm2_args = []

make_stats(p1, p2, ROW_COUNT, COLUMN_COUNT, algorithm, algorithm2, algorithm_args, algorithm2_args, games=100)

Current player: o
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[x][ ][ ][o][ ][ ][ ]
[x][ ][ ][x][ ][ ][ ]
[x][ ][ ][o][ ][ ][ ]
[x][ ][ ][o][ ][x][ ]
Game: 1/100
Moves: 5
Won: x
Current player: o
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[o][ ][x][x][x][x][o]
[x][ ][x][o][o][o][x]
[o][ ][o][o][x][x][x]
[x][o][x][o][x][o][x]
Game: 2/100
Moves: 13
Won: x
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][x][ ][o][x][x][x]
Game: 3/100
Moves: 4
Won: o
Current player: x
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][o][o][o][x][ ]
[ ][o][x][x][x][o][o]
[o][x][x][o][x][x][x]
Game: 4/100
Moves: 10
Won: o
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][x][x][o][ ][ ][ ]
[ ][x][x][o][ ][ ][ ]
Game: 5/100
Moves: 4
Won: o
Current player: x
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[o][ ][ ][o][ ][ ][ ]
[x][o][x][o][x][o][ ]
[o

# Rozgrywka agenta losowego z agentem alpha-beta o głębokości przeszukiwań równej 2

In [31]:
depth = 2
algorithm = RandomSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p1, min_player=p2)
algorithm2 = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p2, min_player=p1)
algorithm_args = [depth, True]
algorithm2_args = [depth, True]

make_stats(p1, p2, ROW_COUNT, COLUMN_COUNT, algorithm, algorithm2, algorithm_args, algorithm2_args, games=100)

Current player: x
[ ][ ][o][ ][ ][ ][ ]
[ ][ ][o][ ][ ][ ][ ]
[ ][ ][o][ ][ ][ ][ ]
[ ][ ][o][ ][ ][ ][ ]
[ ][ ][x][o][x][ ][ ]
[ ][x][x][o][x][x][ ]
Game: 1/100
Moves: 6
Won: o
Current player: x
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][o][x][ ][ ][x]
[o][ ][o][o][ ][ ][x]
[x][ ][x][o][ ][ ][o]
[o][ ][x][o][o][ ][x]
[x][o][x][x][x][o][x]
Game: 2/100
Moves: 12
Won: o
Current player: x
[ ][ ][ ][x][ ][o][ ]
[ ][ ][ ][o][ ][x][ ]
[ ][ ][ ][o][ ][o][x]
[o][x][o][o][o][x][o]
[x][o][o][x][x][o][x]
[o][x][x][o][x][x][x]
Game: 3/100
Moves: 14
Won: o
Current player: x
[ ][ ][ ][ ][o][ ][ ]
[ ][ ][ ][ ][o][ ][ ]
[x][ ][ ][ ][o][ ][x]
[o][ ][ ][ ][x][x][x]
[x][ ][ ][o][o][o][o]
[o][ ][x][o][x][x][x]
Game: 4/100
Moves: 10
Won: o
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[ ][x][x][x][ ][x][ ]
Game: 5/100
Moves: 4
Won: o
Current player: x
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][ ][ ][ ][ ]
[ ][ ][ ][o][ ][ ][ ]
[x][ ][ ][o][ ][ ][ ]
[

# Rozgrywka agenta alpha-beta (depth=2) z agentem wykorzystującym czystą funkcję heurystyczną

In [22]:
depth = 2
algorithm = HeuristicSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p1, min_player=p2)
algorithm2 = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p2, min_player=p1)
algorithm_args = []
algorithm2_args = [depth, True]

make_stats(p1, p2, ROW_COUNT, COLUMN_COUNT, algorithm, algorithm2, algorithm_args, algorithm2_args, games=100)

Current player: x
[x][ ][ ][o][ ][ ][ ]
[o][ ][ ][x][ ][ ][ ]
[o][o][o][o][o][ ][ ]
[x][x][o][x][x][ ][ ]
[o][o][x][o][x][ ][ ]
[x][x][o][x][o][x][x]
Game: 1/10
Moves: 13
Won: o
Current player: x
[x][ ][ ][o][ ][ ][ ]
[o][ ][ ][x][ ][ ][ ]
[o][o][o][o][o][ ][ ]
[x][x][o][x][x][ ][ ]
[o][o][x][o][x][ ][ ]
[x][x][o][x][o][x][x]
Game: 2/10
Moves: 13
Won: o
Current player: x
[o][x][x][o][o][x][o]
[o][o][x][x][o][o][x]
[x][x][o][o][o][x][o]
[o][o][x][x][x][o][x]
[x][x][x][o][o][x][o]
[o][x][o][x][x][o][x]
Game: 3/10
Moves: 21
Current player: x
[ ][ ][x][o][ ][ ][ ]
[o][ ][o][x][ ][ ][ ]
[x][ ][o][o][ ][ ][o]
[o][ ][x][x][o][x][x]
[x][ ][x][o][x][o][o]
[o][x][o][x][o][x][x]
Game: 4/10
Moves: 14
Won: o
Current player: x
[o][o][x][o][o][x][x]
[o][o][o][x][x][o][o]
[x][x][o][o][o][x][o]
[o][o][x][x][x][o][x]
[x][x][x][o][x][x][o]
[o][x][o][x][x][o][x]
Game: 5/10
Moves: 21
Current player: x
[x][ ][ ][o][ ][ ][ ]
[o][ ][ ][x][ ][ ][ ]
[o][o][o][o][o][ ][ ]
[x][x][o][x][x][ ][ ]
[o][o][x][o][x][ ]

# Rozgrywka agenta alpha-beta (depth=4) z agentem wykorzystującym czystą funkcję heurystyczną

In [32]:
depth = 4
algorithm = HeuristicSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p1, min_player=p2)
algorithm2 = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p2, min_player=p1)
algorithm_args = []
algorithm2_args = [depth, True]

make_stats(p1, p2, ROW_COUNT, COLUMN_COUNT, algorithm, algorithm2, algorithm_args, algorithm2_args, games=100)

Current player: x
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][o][ ][o][ ][ ][ ]
[ ][x][o][x][ ][ ][ ]
[ ][o][x][o][ ][ ][ ]
[x][x][o][x][o][ ][x]
Game: 1/100
Moves: 8
Won: o
Current player: x
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][o][ ][o][ ][ ][ ]
[ ][x][o][x][ ][ ][ ]
[ ][o][x][o][ ][ ][ ]
[x][x][o][x][o][ ][x]
Game: 2/100
Moves: 8
Won: o
Current player: x
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][o][x][ ][ ][ ]
[ ][ ][o][o][o][x][ ]
[ ][ ][x][x][o][o][ ]
[ ][ ][x][o][x][o][ ]
[x][x][o][x][o][x][x]
Game: 3/100
Moves: 11
Won: o
Current player: x
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][o][x][ ][ ][ ]
[ ][ ][o][o][o][x][ ]
[ ][ ][x][x][o][o][ ]
[ ][ ][x][o][x][o][ ]
[x][x][o][x][o][x][x]
Game: 4/100
Moves: 11
Won: o
Current player: x
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][o][x][ ][ ][ ]
[ ][ ][o][o][o][x][ ]
[ ][ ][x][x][o][o][ ]
[ ][ ][x][o][x][o][ ]
[x][x][o][x][o][x][x]
Game: 5/100
Moves: 11
Won: o
Current player: x
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][o][x][ ][ ][ ]
[ ][ ][o][o][o][x][ ]
[ ][ ][x][x][o][o][ ]
[

# Rozgrywka agenta mini-max z mini-max (mała głębokość = 2)

In [24]:
depth = 2
algorithm = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p1, min_player=p2)
algorithm2 = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p2, min_player=p1)
algorithm_args = [depth, False]
algorithm2_args =  [depth, False]

make_stats(p1, p2, ROW_COUNT, COLUMN_COUNT, algorithm, algorithm2, algorithm_args, algorithm2_args, games=100)

Current player: o
[o][o][o][ ][o][x][x]
[x][o][o][ ][x][x][o]
[o][o][x][ ][o][o][x]
[x][x][x][x][x][x][o]
[x][o][x][o][o][o][x]
[o][x][o][x][x][x][o]
Game: 1/100
Moves: 19
Won: x
Current player: o
[ ][x][x][ ][ ][ ][o]
[ ][x][o][x][o][ ][x]
[x][o][o][x][x][ ][x]
[o][x][o][o][o][x][o]
[x][o][x][x][x][o][x]
[o][x][o][o][o][x][o]
Game: 2/100
Moves: 17
Won: x
Current player: o
[ ][ ][ ][o][ ][ ][ ]
[ ][ ][ ][x][ ][ ][ ]
[ ][o][ ][o][ ][ ][ ]
[ ][x][ ][x][ ][ ][ ]
[ ][o][o][o][ ][ ][ ]
[x][x][x][x][ ][ ][ ]
Game: 3/100
Moves: 6
Won: x
Current player: x
[x][o][o][x][x][x][o]
[x][o][x][o][o][x][x]
[o][o][o][x][x][o][x]
[x][x][x][o][o][x][o]
[o][o][o][x][x][o][x]
[x][x][o][o][o][x][o]
Game: 4/100
Moves: 21
Current player: x
[o][o][o][x][o][o][o]
[x][x][x][o][x][x][o]
[x][o][x][x][x][o][x]
[o][x][o][o][x][x][o]
[x][o][o][x][o][o][o]
[o][x][x][o][x][x][x]
Game: 5/100
Moves: 21
Current player: x
[o][o][o][x][o][o][o]
[x][x][x][o][x][x][x]
[o][o][x][x][o][o][o]
[x][x][o][o][x][x][x]
[o][o][o][x][o

# Rozgrywka agenta mini-max z mini-max (średnia głębokość = 4)

In [25]:
depth = 4
algorithm = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p1, min_player=p2)
algorithm2 = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p2, min_player=p1)
algorithm_args = [depth, False]
algorithm2_args =  [depth, False]

make_stats(p1, p2, ROW_COUNT, COLUMN_COUNT, algorithm, algorithm2, algorithm_args, algorithm2_args, games=100)

Current player: x
[o][o][x][o][x][ ][o]
[x][o][x][x][o][ ][x]
[x][x][o][o][o][o][o]
[o][o][x][x][o][x][x]
[x][x][o][o][x][o][o]
[x][o][x][x][o][x][x]
Game: 1/100
Moves: 20
Won: o
Current player: x
[o][o][x][o][x][o][o]
[x][o][x][x][o][x][x]
[x][x][o][o][x][o][o]
[o][o][x][x][o][o][x]
[x][x][o][o][x][x][o]
[o][o][x][x][o][x][x]
Game: 2/100
Moves: 21
Current player: x
[o][o][x][o][x][o][o]
[x][o][x][x][o][x][x]
[x][x][o][o][x][o][o]
[o][o][x][x][o][o][x]
[x][x][o][o][x][x][o]
[o][o][x][x][o][x][x]
Game: 3/100
Moves: 21
Current player: x
[o][x][x][o][o][x][o]
[o][o][x][x][x][o][x]
[o][x][o][o][x][x][o]
[x][o][x][x][o][o][x]
[o][x][o][o][x][x][o]
[x][x][o][x][o][o][x]
Game: 4/100
Moves: 21
Current player: x
[x][o][x][o][x][o][x]
[x][o][o][x][x][o][o]
[o][x][x][o][o][x][o]
[x][o][o][x][x][o][x]
[o][x][x][o][o][x][o]
[x][x][o][x][x][o][o]
Game: 5/100
Moves: 21
Current player: x
[o][o][x][o][x][o][o]
[x][x][o][x][x][o][x]
[o][o][x][o][o][x][x]
[x][o][o][x][x][o][o]
[o][x][x][o][o][x][o]
[x][x

# Rozgrywka agenta alpha-beta z alpha-beta (średnia głębokość = 2)

In [26]:
depth = 2
algorithm = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p1, min_player=p2)
algorithm2 = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p2, min_player=p1)
algorithm_args = [depth, True]
algorithm2_args =  [depth, True]

make_stats(p1, p2, ROW_COUNT, COLUMN_COUNT, algorithm, algorithm2, algorithm_args, algorithm2_args, games=100)

Current player: x
[o][o][x][x][o][x][x]
[x][x][o][o][x][o][o]
[x][o][x][x][o][x][x]
[o][x][o][o][x][o][o]
[x][o][x][x][o][x][x]
[o][x][o][o][o][x][o]
Game: 1/100
Moves: 21
Current player: x
[o][o][x][x][o][x][x]
[x][x][o][o][x][o][o]
[x][o][x][x][o][x][x]
[o][x][o][o][x][o][o]
[x][o][x][x][o][x][x]
[o][x][o][o][o][x][o]
Game: 2/100
Moves: 21
Current player: x
[o][o][x][x][o][x][x]
[x][x][o][o][x][o][o]
[x][o][x][x][o][x][x]
[o][x][o][o][x][o][o]
[x][o][x][x][o][x][x]
[o][x][o][o][o][x][o]
Game: 3/100
Moves: 21
Current player: x
[o][o][x][x][o][x][x]
[x][x][o][o][x][o][o]
[x][o][x][x][o][x][x]
[o][x][o][o][x][o][o]
[x][o][x][x][o][x][x]
[o][x][o][o][o][x][o]
Game: 4/100
Moves: 21
Current player: x
[o][o][x][x][o][x][x]
[x][x][o][o][x][o][o]
[x][o][x][x][o][x][x]
[o][x][o][o][x][o][o]
[x][o][x][x][o][x][x]
[o][x][o][o][o][x][o]
Game: 5/100
Moves: 21
Current player: x
[o][o][x][x][o][x][x]
[x][x][o][o][x][o][o]
[x][o][x][x][o][x][x]
[o][x][o][o][x][o][o]
[x][o][x][x][o][x][x]
[o][x][o][o]

# Rozgrywka agenta alpha-beta z alpha-beta (średnia głębokość = 4)

In [33]:
depth = 4
algorithm = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p1, min_player=p2)
algorithm2 = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p2, min_player=p1)
algorithm_args = [depth, True]
algorithm2_args =  [depth, True]

make_stats(p1, p2, ROW_COUNT, COLUMN_COUNT, algorithm, algorithm2, algorithm_args, algorithm2_args, games=10)

Current player: x
[x][x][x][o][o][o][o]
[o][o][x][x][x][o][x]
[o][x][o][o][x][x][o]
[x][o][x][x][o][o][x]
[o][x][o][o][x][x][o]
[x][x][o][x][o][o][x]
Game: 1/10
Moves: 21
Won: o
Current player: x
[x][x][x][o][o][o][o]
[o][o][x][x][x][o][x]
[o][x][o][o][x][x][o]
[x][o][x][x][o][o][x]
[o][x][o][o][x][x][o]
[x][x][o][x][o][o][x]
Game: 2/10
Moves: 21
Won: o
Current player: x
[x][x][x][o][o][o][o]
[o][o][x][x][x][o][x]
[o][x][o][o][x][x][o]
[x][o][x][x][o][o][x]
[o][x][o][o][x][x][o]
[x][x][o][x][o][o][x]
Game: 3/10
Moves: 21
Won: o
Current player: x
[x][x][x][o][o][o][o]
[o][o][x][x][x][o][x]
[o][x][o][o][x][x][o]
[x][o][x][x][o][o][x]
[o][x][o][o][x][x][o]
[x][x][o][x][o][o][x]
Game: 4/10
Moves: 21
Won: o
Current player: x
[x][x][x][o][o][o][o]
[o][o][x][x][x][o][x]
[o][x][o][o][x][x][o]
[x][o][x][x][o][o][x]
[o][x][o][o][x][x][o]
[x][x][o][x][o][o][x]
Game: 5/10
Moves: 21
Won: o
Current player: x
[x][x][x][o][o][o][o]
[o][o][x][x][x][o][x]
[o][x][o][o][x][x][o]
[x][o][x][x][o][o][x]
[o][

# Rozgrywka agenta alpha-beta z alpha-beta (duża głębokość = 6)

In [ ]:
depth = 6
algorithm = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p1, min_player=p2)
algorithm2 = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p2, min_player=p1)
algorithm_args = [depth, True]
algorithm2_args =  [depth, True]

make_stats(p1, p2, ROW_COUNT, COLUMN_COUNT, algorithm, algorithm2, algorithm_args, algorithm2_args, games=10)

# Rozgrywka agenta alpha-beta z alpha-beta (średnia głębokość = 3)

In [29]:
depth = 3
algorithm = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p1, min_player=p2)
algorithm2 = MinMaxSolver(None, ROW_COUNT, COLUMN_COUNT, max_player=p2, min_player=p1)
algorithm_args = [depth, True]
algorithm2_args =  [depth, True]

make_stats(p1, p2, ROW_COUNT, COLUMN_COUNT, algorithm, algorithm2, algorithm_args, algorithm2_args, games=10)

Current player: x
[x][o][x][x][o][o][x]
[x][o][o][o][x][x][o]
[o][o][x][x][o][o][o]
[x][x][o][o][o][x][x]
[x][o][x][x][x][o][x]
[o][x][o][x][o][x][o]
Game: 1/10
Moves: 21
Current player: x
[x][o][x][x][o][o][x]
[x][o][o][o][x][x][o]
[o][o][x][x][o][o][o]
[x][x][o][o][o][x][x]
[x][o][x][x][x][o][x]
[o][x][o][x][o][x][o]
Game: 2/10
Moves: 21
Current player: x
[x][o][x][x][o][o][x]
[x][o][o][o][x][x][o]
[o][o][x][x][o][o][o]
[x][x][o][o][o][x][x]
[x][o][x][x][x][o][x]
[o][x][o][x][o][x][o]
Game: 3/10
Moves: 21
Current player: x
[x][o][x][x][o][o][x]
[x][o][o][o][x][x][o]
[o][o][x][x][o][o][o]
[x][x][o][o][o][x][x]
[x][o][x][x][x][o][x]
[o][x][o][x][o][x][o]
Game: 4/10
Moves: 21
Current player: x
[x][o][x][x][o][o][x]
[x][o][o][o][x][x][o]
[o][o][x][x][o][o][o]
[x][x][o][o][o][x][x]
[x][o][x][x][x][o][x]
[o][x][o][x][o][x][o]
Game: 5/10
Moves: 21
Current player: x
[x][o][x][x][o][o][x]
[x][o][o][o][x][x][o]
[o][o][x][x][o][o][o]
[x][x][o][o][o][x][x]
[x][o][x][x][x][o][x]
[o][x][o][x][o][x

# Podsumowanie
## Mieszane rozgrywki agentów losowych, heurystycznych oraz minimax z przycinaniem $\alpha - \beta$
### Rozgrywka agenta losowego z agentem losowym
Rozgrywka dwóch agentów losowych na planszy o wymiarach 5x7 dla 1000 prób dała rezultat w postaci blisko 50% wygranch gier dla każdego z agentów. Był to wynik spodziewany, ponieważ każdy z agentów losowo wybierał kolumnę z listy dostępnych i nie była zaimplementowana funkcja heurystyczna odpowiadająca za ocenę wpływu ruchu gracza na sytuację na planszy.

### Rozgrywka agenta losowego z agentem wykorzystującym funkcję heurystyczną
W następnym eksperymencie jako przeciwnika gracza losowego wybrano agenta wykorzystującego do wyboru swojego ruchu ocenę planszy funkcją heurystyczną. Jak można było się spodziewać przewaga gracza drugiego znacząco wzrosła o czym świadczą wyniki rozgrywki:
- Próba:                                   100 
- Procent remisów:                          2%
- Procent wygranych gracza 1 (losowy):     16%
- Procent wygranych gracza 2 (heurystyka): 82%

W próbie wystąpiła stosunkowo mała liczba remisów, a procent wygranch gracza pierwszego znacząco spadł. Nie został on jednak zniwelowany do 0%, ponieważ agent 2 wykorzystujący funkcję heurystyczną jest w stanie ocenić jedynie obecną sytuację na planszy (zasymulować ruch i ocenić jego wpływ). Nie ma on możliwości przewidzenia ruchów do przodu tak jak jest to w przypadku algorytmów minimax oraz minimax z przycinaniem $\alpha - \beta$

### Rozgrywka agenta losowego z agentem alpha-beta o głębokości przeszukiwań równej 2
Gracz losowy w starciu z agentem wykorzystującym algorytm minimax z przycinaniem $\alpha - \beta$ o małej głębokości przeszukiwań drzewa gry uzyskał jeszcze gorszy rezultat niż w przypadku starcia z agentem heurystycznym.
Wyniki rozgrywki:
- Próba:                                   100 
- Procent remisów:                          0%
- Procent wygranych gracza 1 (losowy):      9%
- Procent wygranych gracza 2 (heurystyka): 91%

Jak widać procent wygranych gracza drugiego znów wzrósł. Jest to spowodowane dodaniem przewidywania możliwych ruchów w przód dzięki algorytmowi minimax. Nie była to głębokość duża, jednak już pokazała swoją skuteczność. 

### Rozgrywka agenta alpha-beta (depth=2) z agentem wykorzystującym czystą funkcję heurystyczną
Rozgrywka agenta minimax o głębokości przeszukiwań równej 2 z agentem wykorzystującym czystą funkcję heurystyczną dała rezultat w postaci 50% skuteczności każdego z algorytmów, czyli wróciliśmy do sytuacji początkowej, kiedy algorytm losowy uzyskiwał podobne wyniki w starciu z drugim agentem losowym. W tym przypadku jednak algorytm minimax może zyskać jeszcze przewagę, zwiększając głębokość przeszukiwać co pozazano w następnym kroku.

### Rozgrywka agenta alpha-beta (depth=4) z agentem wykorzystującym czystą funkcję heurystyczną
Algorytm minimax z przycinaniem $\alpha - \beta$ o głębokości przeszukiwań równej 4 doprowadził do maksymalizacji swoich zysków uzyskując 100% wygranych w próbie wynoszącej 100 gier w starciu z algorytmem heurystycznym. Widać więc, że jest to algorytm o ogromnym potencjale, czego nie można powiedzieć o funkcji heurystycznej, której możliwość do dalszej konkurencji można poddać w wątpliwość.

## Rozgrywki agentów minimax oraz minimax z przycinaniem $\alpha - \beta$
Ciekawe rozgrywki zaczynają się kiedy dochodzi do starcia agentów minimax z drugim takim samym agentem. Należy w tym momencie zwrócić uwagę na działanie czystego algorytmu minimax, który zakłada przeszukanie całego drzewa gry do pewnej założonej głębokości. Skutkiem tego jest możliwość obrania różnych ścieżek w sytuacji, w której algorytm wyceni dwa lub więcej stanów na tą samą wartość. W tej sytuacji wybiera on losowo jeden z nich. Dlatego zauważalne jest, że kolejne rozgrywki pomiędzy tymi samymi agentami minimax o tych samych parametrach mogą wyglądać różnie i przynieść różny rezultat. Cel algorytmu pozostaje jednak niezmienny i jest nim minimalizacja strat własnych - algorytm dąży do wygranej, a jeżeli nie jest ona możliwa to doprowadza do remisu, co będzie zauważalne w przypadku algorytmu minimax wykorzystującemu algorytm przycinania $\alpha - \beta$. 

Algorytm minimax z przycinaniem $\alpha - \beta$ działa podobnie do zwykłego algorytmu minimax, jednakże w przeciwieństwie do niego nie rozpatruje on dalej gałęzi w drzewie gry o których wie, że na pewno nie zostaną wybrane przez gracza maksymalizującego - parametr $\alpha$ oraz gracza minimalizującego - parametr $\beta$. Przez to przycinanie rozpatruje on średnio o połowę mniej ścieżek. Skutkiem tego jest jednak dochodzenie do jednego możliwego rozwiązania algorytmu, przez co dla kolejnych prób rozgrywek pomiędzy dwoma agentami wykorzystującymi przycinanie $\alpha - \beta$ plansza wygląda tak samo.

### Rozgrywka agenta mini-max z mini-max (mała głębokość = 2)
Dla takiej głębokości przeszukiwań rezultat rozgrywek wygląda następująco:
- Próba:                                   100 
- Procent remisów:                          52%
- Procent wygranych gracza 1 (losowy):      41%
- Procent wygranych gracza 2 (heurystyka): 7%

Jak widać przewagę zyskuje gracz, który wykonuje pierwszy ruch uzyskując 41% wygranych. Drugi gracz uzyskuje rezultat na poziomie 7% wygranch. Co znaczące remisy odpowiadają za 52% wszystkich rozgrywek. Jest to wyraźny skutek działań gracza drugiego, który przez fakt startowania jako drugi, czyli z gorszej pozycji, dąży do remisu, w przypadku jeżeli jego wygrana nie jest możliwa.

### Rozgrywka agenta mini-max z mini-max (mała głębokość = 4)
Dla takiej głębokości przeszukiwań rezultat rozgrywek wygląda następująco:
- Próba:                                   100 
- Procent remisów:                          58%
- Procent wygranych gracza 1 (losowy):      26%
- Procent wygranych gracza 2 (heurystyka): 16%

Zwiększenie głębokości przeszukiwań skutkuje zwiększeniem szans gracza drugiego, który jak już wspomniano, startuje z gorszej pozycji. Zwiększył się jego procent zwycięstw, a co najbardziej zauważalne zmniejszył się procent zwycięstw gracza pierwszego kosztem gracza drugiego i dalszego zwiększania ilości remisów.

### Rozgrywka agenta alpha-beta z alpha-beta (średnia głębokość = 2)
Rozgrywka pomiędzy dwoma agentami wykorzystującymi algorytm minimax z przycinaniem $\alpha - \beta$ wygląda najciekawiej ze wszystkich eksperymentów przeprowadzonych dotychczas i wyglądała ona następująco:
- Próba:                                   100 
- Procent remisów:                         100%
- Procent wygranych gracza 1 (losowy):       0%
- Procent wygranych gracza 2 (heurystyka):   0%

Jak widać doszło do sytuacji w której dla próby wynoszącej 100 rozgrywek żaden z graczy ani razu nie wygrał. Widać, że wszystkie rozgrywki wyglądały w ten sam sposób, co jest skutkiem jak wspomniano brakiem losowości i deterministycznym działaniem algorytmu alpha-beta.

W następnych eksperymentach rozważano inne głębokości przeszukiwań, które tak samo doprowadzały do 100% remisów we wszystkich próbach.